In [1]:
import spacy
from proj_helper import Helper
import pandas as pd

from colorit import *

In [2]:
mybot = Helper()

In [3]:
# item.get_chat_response("Actually I need to check hair cut prices")

In [4]:
def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,
                       truncate_sheet=False, 
                       **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.

    Parameters:
      filename : File path or existing ExcelWriter
                 (Example: '/path/to/file.xlsx')
      df : dataframe to save to workbook
      sheet_name : Name of sheet which will contain DataFrame.
                   (default: 'Sheet1')
      startrow : upper left cell row to dump data frame.
                 Per default (startrow=None) calculate the last row
                 in the existing DF and write to the next row...
      truncate_sheet : truncate (remove and recreate) [sheet_name]
                       before writing DataFrame to Excel file
      to_excel_kwargs : arguments which will be passed to `DataFrame.to_excel()`
                        [can be dictionary]

    Returns: None

    """
    from openpyxl import load_workbook

    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='openpyxl')

    # Python 2.x: define [FileNotFoundError] exception if it doesn't exist 
    try:
        FileNotFoundError
    except NameError:
        FileNotFoundError = IOError


    try:
        # try to open an existing workbook
        writer.book = load_workbook(filename)
        
        # get the last row in the existing Excel sheet
        # if it was not specified explicitly
        if startrow is None and sheet_name in writer.book.sheetnames:
            startrow = writer.book[sheet_name].max_row

        # truncate sheet
        if truncate_sheet and sheet_name in writer.book.sheetnames:
            # index of [sheet_name] sheet
            idx = writer.book.sheetnames.index(sheet_name)
            # remove [sheet_name]
            writer.book.remove(writer.book.worksheets[idx])
            # create an empty sheet [sheet_name] using old index
            writer.book.create_sheet(sheet_name, idx)
        
        # copy existing sheets
        writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    except FileNotFoundError:
        # file does not exist yet, we will create it
        pass

    if startrow is None:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, **to_excel_kwargs)

    # save the workbook
    writer.save()

In [5]:
def savebotchats():
    rating = {'sessionID': [bot_session_id], 
              'rating': [session_rating]}
    df1 = pd.DataFrame(data=rating)
    append_df_to_excel('chatbot_rating.xlsx', df1)
    
    history = {'sessionID': sessionID * len(botchat), 
               'userchat': userchat,
               'botchat': botchat,
               'predgroup': predgroup}
    df2 = pd.DataFrame(data=history)
    append_df_to_excel('chatbot_chat_history.xlsx', df2)

In [6]:
bot_session_id = len(pd.read_excel('chatbot_rating.xlsx', header=0))

botchat = []
userchat = []
predgroup = []
sessionID = [bot_session_id]
session_rating = 0

In [ ]:
stop_chatbot = False
# Use this to ensure that ColorIt will be usable by certain command line interfaces
init_colorit()

print(color("Sorry, I'm not available now. Chatbot will help you for service check, put/change/cancel appointments.", Colors.blue),'\n',
     color("* Please make sure to ask one service or request at once.", Colors.blue),'\n',
     color("* Chat will be auto closed after one sucessful request for put/change/remove appointment.", Colors.blue),'\n',
     color("* Please use a seperate chat, If you have multiple appoitment requests.", Colors.blue),'\n',
     color("* Type 'EXIT' if it's not responding to your requests.", Colors.blue),'\n')
while not stop_chatbot:
    user_input = input("Your Reply: ")
    userchat.append(user_input)
    
    if user_input == 'EXIT':
        stop_chatbot = True
        predgroup.append(0)
        botchat.append('EXIT_BOT')
        
        session_rating = 0 # EXIT Due to interuption
        savebotchats()
        
    else:
        bot_reply = mybot.get_chat_response(user_input)
        predgroup.append(mybot.current_group)
        botchat.append(bot_reply)
        
        print('\n',color('Chatbot Reply: ', Colors.red),color(bot_reply, Colors.red),'\n')
        if mybot.is_exit_chat == True:
            stop_chatbot = True
            session_rating = int(input("Please rate Chatbot to scale 1-5. \(1 - unsatisfied, .. 5 - Highly Satisfied)\: "))
            savebotchats()

Sorry, I'm not available now. Chatbot will help you for service check, put/change/cancel appointments. 
 * Please make sure to ask one service or request at once. 
 * Chat will be auto closed after one sucessful request for put/change/remove appointment. 
 * Please use a seperate chat, If you have multiple appoitment requests. 
 * Type 'EXIT' if it's not responding to your requests. 

Your Reply: Hi How are you?

 Chatbot Reply:  Good Evening I'm doing fine ! :)  How can I help you? 

Your Reply: I need to know about men haircut

 Chatbot Reply:  We have special haircut styles for Men. From $ 20 

Your Reply: Okay, I need an appointment

 Chatbot Reply:  Awesome ! For the appointment inquiry process, I just need some details from you :)First I need the person name for the appointment :) 

Your Reply: Appointment for Chathuranga

 Chatbot Reply:  Ohh okay :) So the appointment for Chathuranga? Right ? :) 

Your Reply: yes

 Chatbot Reply:  Okay. Now could you please provide preferred da

References <br>
01. https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/ <br>
02. Added interjections <br>
03. https://stackoverflow.com/questions/20219254/how-to-write-to-an-existing-excel-file-without-overwriting-data-using-pandas